In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


usaid = pd.read_csv('/Users/vibhuverma/Desktop/CLASSWORK/BUSINESS PRACTICUM/Anti-Corruption/Data Provided/USAID_Project_Data/USAID_Anticorruption_Projects_Database.csv')

usaid.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

df = usaid[['proj_desc']]



In [4]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


In [5]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [6]:
#Tokenize each word

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long_term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic_republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking', 'and

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['component', 'program', 'seek', 'take', 'already', 'exist', 'improve', 'implement', 'find', 'way', 'ensure', 'progress', 'achieve', 'sustainable', 'project', 'help', 'government', 'islamic_republic', 'strengthen', 'high', 'office', 'oversight', 'hoo', 'make', 'strong', 'effective', 'institution', 'able', 'lead', 'monitor', 'coordinate', 'report', 'effort', 'combat', 'corruption', 'country', 'project', 'work', 'implement', 'program', 'institutional', 'development', 'sustainability', 'also', 'support', 'office', 'carry', 'priority', 'asset', 'registration', 'verification', 'complaint', 'management', 'case', 'track', 'coordination', 'monitor', 'anticorruption', 'performance', 'government', 'agency']]


In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 2), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]]


In [13]:
#TF-IDF
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.stem.porter import *
from gensim import corpora, models



tfidf = models.TfidfModel(corpus)

#transformation to the entire corpus and calling it ‘corpus_tfidf’

corpus_tfidf = tfidf[corpus]

In [14]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus_tfidf]

[(0,
  '0.022*"sustain" + 0.019*"child" + 0.018*"maternal" + 0.018*"bhs" + '
  '0.004*"model" + 0.002*"south" + 0.002*"popular" + 0.002*"mobilization" + '
  '0.002*"uphold" + 0.001*"watchdog"'),
 (1,
  '0.027*"respect" + 0.016*"prepare" + 0.015*"impact" + 0.013*"agenda" + '
  '0.011*"tool" + 0.008*"test" + 0.007*"testing" + 0.007*"human_right" + '
  '0.006*"threshold" + 0.006*"care"'),
 (2,
  '0.005*"cross" + 0.003*"solve" + 0.003*"drafting" + 0.003*"cut" + '
  '0.001*"aid" + 0.001*"know" + 0.001*"direct" + 0.001*"integrated" + '
  '0.001*"subnational" + 0.001*"jsdp"'),
 (3,
  '0.006*"reinforce" + 0.006*"many" + 0.003*"trafficking" + 0.002*"province" + '
  '0.001*"end" + 0.001*"transformation" + 0.001*"record" + 0.001*"region" + '
  '0.001*"poppy" + 0.001*"product"'),
 (4,
  '0.017*"period" + 0.014*"incentive" + 0.011*"water" + 0.008*"gender" + '
  '0.008*"voter" + 0.007*"secure" + 0.006*"account" + 0.005*"wastewater" + '
  '0.005*"launch" + 0.004*"face"'),
 (5,
  '0.024*"document" + 0

In [17]:

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.426415445707907

Coherence Score:  0.41119818686934406


In [18]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.350354  0.007924       1        1  56.348507
17     0.062650 -0.182361       2        1  12.689342
8      0.074185  0.200550       3        1  11.289804
19     0.039259 -0.009596       4        1   6.213360
16     0.025433 -0.004093       5        1   3.012430
12     0.021799 -0.003218       6        1   2.374453
15     0.017438 -0.002231       7        1   1.743281
11     0.014586 -0.001661       8        1   1.163190
1      0.012335 -0.001232       9        1   0.890978
4      0.009716 -0.000758      10        1   0.759886
5      0.011510 -0.001084      11        1   0.748766
14     0.012297 -0.001229      12        1   0.679611
7      0.008817 -0.000591      13        1   0.442476
10     0.007475 -0.000359      14        1   0.440250
18     0.007443 -0.000350      15        1   0.435525
0      0.008962 -0.000613      16        1   0.283241
3      0.004357  0.000180      17        1   0.165259
2      0.004124  0.000221      18        1   0.151692
6      0.004704  0.000127      19        1   0.113367
13     0.003265  0.000371      20        1   0.054589, topic_info=              Term       Freq      Total Category  logprob  loglift
633         health  67.000000  67.000000  Default  30.0000  30.0000
308         reform  64.000000  64.000000  Default  29.0000  29.0000
231     democratic  55.000000  55.000000  Default  28.0000  28.0000
191         design  45.000000  45.000000  Default  27.0000  27.0000
83            rule  44.000000  44.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
670  globalization   0.004084   2.630744  Topic20  -7.5843   1.0452
671          human   0.004084  17.404884  Topic20  -7.5843  -0.8443
672       judicial   0.004084  75.733421  Topic20  -7.5843  -2.3148
673        account   0.004084   2.989561  Topic20  -7.5843   0.9173
646           tool   0.004084   3.701550  Topic20  -7.5843   0.7037

[991 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
100       1  0.338834    academic
100       7  0.338834    academic
995       1  0.384344  accessible
748       1  0.428897   accession
673       1  0.334497     account
...     ...       ...         ...
142       1  0.037573       woman
142       2  0.939323       woman
182       1  0.098170       would
182       4  0.785359       would
1426      1  0.428891       write

[975 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 18, 9, 20, 17, 13, 16, 12, 2, 5, 6, 15, 8, 11, 19, 1, 4, 3, 7, 14])

In [ ]:
##TUNING

# Tuning - results

def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()





import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus_tfidf)
corpus_sets = [gensim.utils.ClippedCorpus(corpus_tfidf, num_of_docs*0.75), 
               corpus_tfidf]
corpus_title = ['75% corpus_tfidf', '100% corpus_tfidf']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

for i in range(len(corpus_sets)):
        # iterate through number of topics
    for k in topics_range:
            # iterate through alpha values
        for a in alpha:
                # iterare through beta values
            for b in beta:
                    # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dictionary, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                model_results['Validation_Set'].append(corpus_title[i])
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                    
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    
